In [1]:
import os
from langchain_google_genai.llms import GoogleGenerativeAI
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate
from langchain_core.runnables.passthrough import RunnablePassthrough
from langchain.memory.buffer import ConversationBufferMemory
from dotenv import load_dotenv, find_dotenv

In [2]:
load_dotenv(find_dotenv())

True

In [3]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY", "")

In [17]:
EXAMPLES = [
    {
        "query": "Product build for consumers which needs collection of personal data of the user.",
        "answer": "Data security and Privacy rights.",
        "explanation": """
          As collection of personal data is there and security of data is require we need Privacy Rights and Data Security GDPR guidelies.
        """
    },
    {
        "query": "Product which requires collection of personal data and sends to third parties.",
        "answer": "Data security, Privacy rights, Lawful basis and transparency, and Accountability and governance",
        "explanation": """
        In collection of personal data and security of collected data, Privacy Rights and Data Security guidelines is needed. As it 
        also sends to third party we need to follow Lawful basis and transparency, and Accountability and governance to make a agreement 
        between third parties and our organisation and a person to oversee it.
        """
    }
]

In [18]:
prompt = PromptTemplate(
    template="""
    Role: You are an Expert in General Data Protection Regulation (GDPR) guidelines which tells which guideline(s) 
          to follow when building an application or product.
    Instructions: Answer user's query and outputs which guide line to follow. Take help from provided examples and 
    don't make up any answer from your side. First completely analyse the question which is in backticks and its associated
    answer and explanation in <> brackets. Properly think from your side before giving answers and only provide guideline name(s) in below format
    GuideLines: 


    
    Question: ``` {query} ```
    Answer: < {answer} >
    Explanation: < {explanation} >
    """,
    input_variables=["query", "answer", "explanation"]
)

In [19]:
few_shot_template = FewShotPromptTemplate(
    examples=EXAMPLES,
    example_prompt=prompt,
    suffix="Question: {query}",
    input_variables=["query"]
)

In [20]:
print(few_shot_template.format(query="Ask any question"))


    Role: You are an Expert in General Data Protection Regulation (GDPR) guidelines which tells which guideline(s) 
          to follow when building an application or product.
    Instructions: Answer user's query and outputs which guide line to follow. Take help from provided examples and 
    don't make up any answer from your side. First completely analyse the question which is in backticks and its associated
    answer and explanation in <> brackets. Properly think from your side before giving answers and only provide guideline name(s) in below format
    GuideLines: 


    
    Question: ``` Product build for consumers which needs collection of personal data of the user. ```
    Answer: < Data security and Privacy rights. >
    Explanation: < 
          As collection of personal data is there and security of data is require we need Privacy Rights and Data Security GDPR guidelies.
         >
    


    Role: You are an Expert in General Data Protection Regulation (GDPR) guideline

In [21]:
google_llm = GoogleGenerativeAI(
    model="gemini-1.5-pro-latest", 
    temperature=0.3
)

In [22]:
chain = (
    {"query": RunnablePassthrough()}
    | few_shot_template
    | google_llm
)

In [23]:
ans = chain.invoke(
    """
    We are building an online book store through which person can order book. For that we collect 
    personal information like person's name, address, phone number. Let me know which GDPR guidelines I need to follow?
    """
)

print(ans)

GuideLines: Data security, Privacy rights, Lawful basis and transparency 



In [24]:
result = chain.invoke(
    """
    A product which collects personal information like person's name, address, phone number and sends to third party for data processing.    
    Suggest the guideline names which I need to follow?
    """
)

print(result)

GuideLines: 
Data security, Privacy rights, Lawful basis and transparency, and Accountability and governance 



In [25]:
res = chain.invoke(
    """
    A web application which needs some general data and store it on its server. Which GDPR guidelines I need here?
    """
)

print(res)

GuideLines: Data security, Privacy rights 



In [26]:
res1 = chain.invoke(
    """
    A web application hosted on some third party cloud service and stores the data. Tell me which GDPR guidelines are needed here?
    """
)

print(res1)

GuideLines: 
Data security, Accountability and governance 



In [28]:
res2 = chain.invoke(
    """
    A backend application which makes API calls to get the personal data and process it on third party cloud service
    but doesn't store it. Tell me which GDPR guidelines are needed here?
    """
)

print(res2)

GuideLines: Data security, Privacy rights, Lawful basis and transparency, and Accountability and governance 

